In [2]:
import pandas as pd

keywords_by_sentiment = {
    2: ["rất tốt","tài","cao","giỏi", "tốt", "xuất sắc", "đáng khen", "hiệu quả", "tích cực", "nhiệt tình", "chủ động"],
    1: ["tạm ổn", "chưa ổn định", "cũng được", "ở mức chấp nhận được", "bình thường", "trung bình", "đôi lúc tốt", "không rõ ràng"],
    0: ["tệ", "tồi","thấp", "lười","kém", "thiếu trách nhiệm", "không tốt", "rất kém", "bị phàn nàn", "tiêu cực"]
}

prefix_phrases = [
    "Thái độ", "Hiệu suất", "Cách làm việc", "Tinh thần", "Chất lượng công việc",
    "Phản hồi", "Ý thức", "Tác phong", "Thói quen làm việc", "Kết quả thực hiện"
]

context_phrases = [
    "trong công việc", "trong quá trình xử lý", "khi làm nhóm", "khi thực hiện dự án",
    "trong môi trường áp lực", "ở giai đoạn thử việc", "khi giao tiếp với đồng nghiệp",
    "trong thời gian gần đây", "khi được giao nhiệm vụ", "với khách hàng"
]

rows = []
seen = set()
start_id = 30000

for sentiment, keywords in keywords_by_sentiment.items():
    for prefix in prefix_phrases:
        for keyword in keywords:
            for context in context_phrases:
                sentence = f"{prefix} {keyword} {context}"
                if sentence not in seen:
                    rows.append({
                        "ID": f"gen_{start_id + len(rows)}",
                        "comment": sentence,
                        "sentiment": sentiment
                    })
                    seen.add(sentence)

df = pd.DataFrame(rows)
df.to_csv("sentiment_data.csv", index=False, encoding="utf-8")
print(f"Tạo thành công {len(df)} dòng.")


Tạo thành công 2900 dòng.


In [4]:
!pip install underthesea flask pyngrok scikit-learn pandas requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00


# Mục mới

# Mục mới

In [5]:
from pyngrok import ngrok

# Kill tất cả các tunnel đang chạy
ngrok.kill()


In [6]:
import pandas as pd
import re
import underthesea
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import logging
from flask import Flask, request, jsonify
import threading
from pyngrok import ngrok, conf
import requests
import json
import numpy as np

# Thiết lập logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Khởi tạo Flask app
app = Flask(__name__)

# Thiết lập ngrok authtoken
NGROK_AUTH_TOKEN = "2zcxV64269qISWB9CHZi57UIIo4_5yKJHM6ZvGiyTPqkUUg4o"
try:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    print("Ngrok authtoken set successfully")
except Exception as e:
    logger.error(f"Error setting ngrok authtoken: {str(e)}")
    raise

# Hàm tiền xử lý văn bản tiếng Việt
def preprocess_text(text):
    try:
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = underthesea.word_tokenize(text, format="text")
        return text
    except Exception as e:
        logger.error(f"Error in preprocess_text: {str(e)}")
        return text

# Tải và huấn luyện mô hình từ file CSV
try:
    # Đọc file CSV
    train_data = pd.read_csv('sentiment_data.csv', encoding='utf-8')

    # Kiểm tra dữ liệu
    print(f"Sample data: {train_data.head().to_dict()}")
    print(f"Sentiment labels: {train_data['sentiment'].unique()}")

    # Tiền xử lý dữ liệu huấn luyện
    train_data['text'] = train_data['comment'].apply(preprocess_text)

    # Vector hóa và huấn luyện mô hình
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train = vectorizer.fit_transform(train_data['text'])
    y_train = train_data['sentiment']  # Sử dụng nhãn văn bản trực tiếp
    model = MultinomialNB()
    model.fit(X_train, y_train)
    print("Naive Bayes model trained successfully")
except Exception as e:
    logger.error(f"Error in training model: {str(e)}")
    raise

# Hàm phân tích cảm xúc
def analyze_sentiment(text):
    try:
        processed_text = preprocess_text(text)
        vectorized_text = vectorizer.transform([processed_text])
        prediction = model.predict(vectorized_text)[0]
        return str(prediction)
    except Exception as e:
        return "-1"

# Route API để phân tích cảm xúc
@app.route('/sentiment-analysis', methods=['POST'])
def evaluate_employee():
    try:
        data = request.get_json()
        if not data or 'comment' not in data:
            logger.error("Invalid input:'comment' missing")
            return jsonify({'error': 'Dữ liệu đầu vào không hợp lệ, cần chứa "comment"'}), 400

        comment = data['comment']

        # Phân tích cảm xúc
        sentimentRaw = int(analyze_sentiment(comment))  # chuyển về số nguyên
        sentiment = ""
        if sentimentRaw == 0:
            sentiment = "Chưa tốt"
        elif sentimentRaw == 1:
            sentiment = "Trung bình"
        elif sentimentRaw == 2:
            sentiment = "Tốt"
        else:
            sentiment = "Không xác định"

        # Tạo kết quả đầu ra
        result = {
            'sentiment': sentiment
        }

        print(f"Sentiment analysis completed for sentiment: {sentiment}")
        return jsonify(result), 200
    except Exception as e:
        logger.error(f"Error in evaluate_employee: {str(e)}")
        return jsonify({'error': str(e)}), 500

# Hàm chạy Flask
def run():
    app.run(host='0.0.0.0', port=5555)

# Tạo public URL và chạy
try:
    public_url = ngrok.connect(5555, bind_tls=True).public_url
    print(f"Ngrok URL: {public_url}")
except Exception as e:
    logger.error(f"Error starting ngrok: {str(e)}")
    raise

# Khởi động Flask trong thread riêng
thread = threading.Thread(target=run)
thread.start()



Ngrok authtoken set successfully
Sample data: {'ID': {0: 'gen_30000', 1: 'gen_30001', 2: 'gen_30002', 3: 'gen_30003', 4: 'gen_30004'}, 'comment': {0: 'Thái độ rất tốt trong công việc', 1: 'Thái độ rất tốt trong quá trình xử lý', 2: 'Thái độ rất tốt khi làm nhóm', 3: 'Thái độ rất tốt khi thực hiện dự án', 4: 'Thái độ rất tốt trong môi trường áp lực'}, 'sentiment': {0: 2, 1: 2, 2: 2, 3: 2, 4: 2}}
Sentiment labels: [2 1 0]
Naive Bayes model trained successfully
Ngrok URL: https://da74df81321e.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


In [1]:
import requests
import json
import logging

# Thiết lập logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Test API
try:
    url = public_url + "/sentiment-analysis"
    test_data = {
        "comment": "làm việc đạt năng suất cao"
    }
    response = requests.post(url, json=test_data)

    # Kiểm tra mã trạng thái HTTP
    if response.status_code == 200:
        print(json.dumps(response.json(), ensure_ascii=False, indent=2))
    else:
        logger.error(f"API request failed with status {response.status_code}: {response.text}")
        print(f"Error: API returned status {response.status_code}: {response.text}")

except Exception as e:
    logger.error(f"Error in API test: {str(e)}")
    print(f"Error in API test: {str(e)}")

ERROR:__main__:Error in API test: name 'public_url' is not defined


Error in API test: name 'public_url' is not defined
